# Quantum State Tomography

In [1]:
import qiskit
import qiskit_experiments as qe
tomo = qe.tomography

# For simulation
from qiskit.providers.aer import AerSimulator
from qiskit.test.mock import FakeParis

# Noisy simulator backend
backend = AerSimulator.from_backend(FakeParis())

## State Tomography Experiment

To run a state tomography experiment we initialize the experiment with a circuit to prepare the state to be measured. We can also pass in an `Operator`, or a `Statevector` to describe the preparation circuit.

In [2]:
# Run experiments

# GHZ State preparation circuit
nq = 2
qc_ghz = qiskit.QuantumCircuit(nq)
qc_ghz.h(0)
qc_ghz.s(0)
for i in range(1, nq):
    qc_ghz.cx(0, i)

# QST Experiment
qstexp1 = tomo.StateTomography(qc_ghz)
qstdata1 = qstexp1.run(backend, seed_simulation=100)
qstresult = qstdata1.analysis_result(-1)

print('FIT RESULT')
for key, val in qstresult.items():
    print(f'{key}:', val)

FIT RESULT
state: DensityMatrix([[ 4.87955729e-01+0.j        ,  6.99869792e-03-0.00585938j,
                -3.25520833e-04-0.02490234j,  5.37109375e-03-0.44775391j],
               [ 6.99869792e-03+0.00585938j,  3.38541667e-02+0.j        ,
                -4.39453125e-03-0.00537109j,  3.58072917e-03-0.00830078j],
               [-3.25520833e-04+0.02490234j, -4.39453125e-03+0.00537109j,
                 2.37630208e-02+0.j        ,  1.28580729e-02-0.01953125j],
               [ 5.37109375e-03+0.44775391j,  3.58072917e-03+0.00830078j,
                 1.28580729e-02+0.01953125j,  4.54427083e-01+0.j        ]],
              dims=(2, 2))
fitter: linear_inversion
fitter_time: 0.002052783966064453
state_eigvals: [0.92046815 0.04229466 0.03198052 0.00525667]
state_trace: 1.0000000000000004
value: 0.9189453125000002
value_label: state_fidelity
success: True


### Tomography Results

The main results for tomography are the fitted state, which is stored in the `"state"` key as a `DensityMatrix` object:

In [3]:
print(qstresult["state"])

DensityMatrix([[ 4.87955729e-01+0.j        ,  6.99869792e-03-0.00585938j,
                -3.25520833e-04-0.02490234j,  5.37109375e-03-0.44775391j],
               [ 6.99869792e-03+0.00585938j,  3.38541667e-02+0.j        ,
                -4.39453125e-03-0.00537109j,  3.58072917e-03-0.00830078j],
               [-3.25520833e-04+0.02490234j, -4.39453125e-03+0.00537109j,
                 2.37630208e-02+0.j        ,  1.28580729e-02-0.01953125j],
               [ 5.37109375e-03+0.44775391j,  3.58072917e-03+0.00830078j,
                 1.28580729e-02+0.01953125j,  4.54427083e-01+0.j        ]],
              dims=(2, 2))


Additional data stored in the tomography result including:
- `"value"`: The `state_fidelity` with the ideal target state
- `"state_trace"`: The trace of the fitted state (sum of state eigenvalues)
- `"state_eigvals"`: the eigenvalues of the fitted state

In [4]:
print("State fidelity =", qstresult["value"])
print("State eigenvalues =", qstresult["state_eigvals"])
print("State trace =", qstresult["state_trace"])

State fidelity = 0.9189453125000002
State eigenvalues = [0.92046815 0.04229466 0.03198052 0.00525667]
State trace = 1.0000000000000004


#### Additional result metadata

If the fitted returned a non-positive hat was rescaled to be PSD there will be additional fields in the metadata containing the raw (unscaled) eigenvalues:

- `"raw_eigenvalues"`: the eigenvalues of the state before rescaling
- `"rescaled_psd:` : True if PSD rescaling was performed.

If trace rescaling was performed because the input state was not trace 1, there will also be extra fields:
- `"raw_trace"`: the sum of the raw eigenvalues before rescaling
- `"rescaled_trace"`: True if trace rescaling was performed

To see the effect of rescaling we can perform a "bad" fit with very low counts

In [5]:
# QST Experiment
bad_data = qstexp1.run(backend, shots=10, seed_simulation=100)
bad_result = bad_data.analysis_result(-1)

print('FIT RESULT')
for key, val in bad_result.items():
    print(f'{key}:', val)

FIT RESULT
state: DensityMatrix([[ 0.48466135+0.00000000e+00j,  0.0351084 +5.00191453e-02j,
                -0.06052864-2.99669459e-02j,  0.00565657-3.99468876e-01j],
               [ 0.0351084 -5.00191453e-02j,  0.02085624+3.25260652e-19j,
                -0.02773083+2.44702872e-02j, -0.0066963 -3.71313180e-02j],
               [-0.06052864+2.99669459e-02j, -0.02773083-2.44702872e-02j,
                 0.0722317 +0.00000000e+00j, -0.0403584 +9.04506868e-03j],
               [ 0.00565657+3.99468876e-01j, -0.0066963 +3.71313180e-02j,
                -0.0403584 -9.04506868e-03j,  0.42225071+0.00000000e+00j]],
              dims=(2, 2))
fitter: linear_inversion
fitter_time: 0.0014231204986572266
rescaled_psd: True
state_eigvals: [0.8630812 0.1369188 0.        0.       ]
raw_eigvals: [ 0.96091694  0.23475455 -0.03939899 -0.1562725 ]
state_trace: 0.9999999999999984
value: 0.8529249072196502
value_label: state_fidelity
success: True


## Tomography Fitters

The default fitters is `linear_inversion`, which reconstructs the state using *dual basis* of the tomography basis. This will typically result in a non-postive reconstructed state. This state is rescaled to be postive-semidfinite (PSD) by computing its eigen-decomposition and rescaling its eigenvalues using the approach from *J Smolin, JM Gambetta, G Smith, Phys. Rev. Lett. 108, 070502 (2012), [open access](https://arxiv.org/abs/arXiv:1106.5458).

There are several other fitters are included (See API documentation for details). For example if `cvxpy` is installed we can use the `cvxpy_gaussian_lstsq` fitter which allows constraining the fit to be PSD without requiring rescaling.

In [6]:
qstexp1.run_analysis(qstdata1, fitter='cvxpy_linear_lstsq')
qstresult2 = qstdata1.analysis_result(-1)

print('FIT RESULT')
for key, val in qstresult2.items():
    print(f'{key}:', val)

FIT RESULT
state: DensityMatrix([[ 4.87955754e-01+0.00000000e+00j,
                 6.99870000e-03-5.85933455e-03j,
                -3.25598752e-04-2.49023865e-02j,
                 5.37110228e-03-4.47754024e-01j],
               [ 6.99870000e-03+5.85933455e-03j,
                 3.38541760e-02+0.00000000e+00j,
                -4.39458616e-03-5.37112965e-03j,
                 3.58076870e-03-8.30077829e-03j],
               [-3.25598752e-04+2.49023865e-02j,
                -4.39458616e-03+5.37112965e-03j,
                 2.37629662e-02-1.73472348e-18j,
                 1.28580366e-02-1.95313352e-02j],
               [ 5.37110228e-03+4.47754024e-01j,
                 3.58076870e-03+8.30077829e-03j,
                 1.28580366e-02+1.95313352e-02j,
                 4.54427104e-01+0.00000000e+00j]],
              dims=(2, 2))
cvxpy_solver: CVXOPT
cvxpy_status: optimal
fitter: cvxpy_linear_lstsq
fitter_time: 0.06333804130554199
rescaled_trace: True
state_eigvals: [0.92046829 0.04229472 0.03

## Parallel Tomography Experiment

We can also use the `qiskit_experiments.ParallelExperiment` class to run subsystem tomography on multiple qubits in parallel.

For example if we want to perform 1-qubit QST on several qubits at once:

In [7]:
from math import pi
num_qubits = 5
gates = [qiskit.circuit.library.RXGate(i * pi / (num_qubits - 1))
         for i in range(num_qubits)]

subexps = [
    tomo.StateTomography(gate, qubits=[i])
    for i, gate in enumerate(gates)
]
parexp = qe.composite.ParallelExperiment(subexps)
pardata = parexp.run(backend, seed_simulation=100)
print(pardata.analysis_result(-1))


- experiment_types: ['StateTomography', 'StateTomography', 'StateTomography', 'StateTomography', 'StateTomography']
- experiment_ids: ['a74e8105-c3a4-48a9-8523-82d5999a6337', '49db1987-2719-436d-b447-ffb52d70ef54', 'ee4a0325-938f-476a-8d0e-bc0422cf8747', 'e1e02742-7e44-40b8-aa51-d0da07f3f094', '2b62d974-ab35-4b81-b595-b64322f637cf']
- experiment_qubits: [(0,), (1,), (2,), (3,), (4,)]
- success: True


View component experiment analysis results

In [8]:
for i in range(parexp.num_experiments):
    expdata = pardata.component_experiment_data(i)
    result = expdata.analysis_result(-1)
    
    print(f'\nPARALLEL EXP {i}: FIT RESULT')
    for key, val in result.items():
        print(f'{key}:', val)


PARALLEL EXP 0: FIT RESULT
state: DensityMatrix([[0.98144531+0.j        , 0.05761719-0.00292969j],
               [0.05761719+0.00292969j, 0.01855469+0.j        ]],
              dims=(2,))
fitter: linear_inversion
fitter_time: 0.00013399124145507812
state_eigvals: [0.98488959 0.01511041]
state_trace: 1.0
value: 0.9814453124999999
value_label: state_fidelity
success: True

PARALLEL EXP 1: FIT RESULT
state: DensityMatrix([[0.85058594+0.j        , 0.01074219+0.31445313j],
               [0.01074219-0.31445313j, 0.14941406+0.j        ]],
              dims=(2,))
fitter: linear_inversion
fitter_time: 0.00010800361633300781
state_eigvals: [0.9710697 0.0289303]
state_trace: 0.9999999999999998
value: 0.9702536308476938
value_label: state_fidelity
success: True

PARALLEL EXP 2: FIT RESULT
state: DensityMatrix([[0.51953125+0.j        , 0.01464844+0.48242188j],
               [0.01464844-0.48242188j, 0.48046875+0.j        ]],
              dims=(2,))
fitter: linear_inversion
fitter_time: 0.0001